<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/embedding_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Agent/Cores/agent_client.py
import requests

class Agent_Client:
    def __init__(self,
                 url_path='http://127.0.0.1:9000/'):
        r"""Create Client instance that will post message to server.

        :param url_path: The Server Url path.

        :return: `Agent_Client` instance.
        """
        self.url_path = url_path

    def post(self, func_name='translate', data=None):
        r"""Sends message to server to run a function.

        :param func_name: function name that need to sever excute.

        :return: string response.
        """
        res = requests.post(url=self.url_path + func_name, json=data)
        return res

In [ ]:
#Agent/Cores/agent_server.py
from cores.server import *
from cores.install_packages import *

class Agent_Server:
    class_agent = None
    def __init__(self,
                ip="127.0.0.1",
                port=9000,
                obj=None,
                is_install_offline_package=False,
                 ):
        r"""Creates Server instance that listen message from client.

        :param ip: binding ip. (default: 127.0.0.1)
        :param port: binding port. (default: 9000)
        :param obj: object tasks
        :param is_install_offline_package: set True if need to newly install.

        :return: `Agent_Server` instance.
        """
        if is_install_offline_package:
            install_whl('packages')
        self.server = Server_Cores(ip=ip, port=port)
        Agent_Server.class_agent = self.server
        self.obj = obj

    def run(self):
        r"""Starts Agent.
        """
        self.server.add_all_functions_to_rule(self.obj)
        self.server.run()

    def shutdown(self):
        r"""Shutdowns Agent.
        """
        self.server.shutdown_server()

    @classmethod
    def get_data(self):
        r"""Gets requested data from client.
        """
        if Agent_Server.class_agent:
            return Agent_Server.class_agent.get_request_data()
        else:
            return ""


In [ ]:
#Agent/Cores/server.py
from flask import Flask
from flask import request
from inspect import getmembers, ismethod

class Server_Cores():
    def __init__(self,
                 ip='127.0.0.1',
                 port=9000,
                 name=__name__,
                 ):
        r"""Register a rule for routing incoming requests and building URLs.

        :param ip: Web services IP.
        :param port: Web services Port.
        :param name: Web services Port.

        :return: `Server_Cores` instance object.
        """
        try:
            self.ip = ip
            self.port = port
            self.app = Flask(name)
            self.is_err = False
            self.err_msg = ""
            # self.add_url_rule('hello', view_func=self.hello)
        except BaseException as ex:
            self.app = None
            self.is_err = True
            self.err_msg = str(ex)
            print(str(ex))


    def add_url_rule(self, rule, view_func=None):
        r"""Register a rule for routing incoming requests and building URLs.

        :param endpoint: The URL rule string.
        :param view_func: (ft.RouteCallable | None) – The view function to associate with the endpoint name.

        :return: False if any error occur. otherwise return True
        """
        try:
            options = {'methods':['POST']}
            rule_append = f"/{rule}"
            self.app.add_url_rule(rule=rule_append, view_func=view_func, **options)
            return True
        except BaseException as e:
            self.is_err = True
            self.err_msg = str(e)
            return False


    def hello(self, a=""):
        aa = a
        return "hello world " + a


    def run(self):
        """
        Runs web server.
        """
        if (self.app):
            self.app.run(self.ip, self.port)
        else:
            print('Server is null. Please create a intance of server before running.')


    def shutdown_server(self):
        """
        Shutdown Server.
        """
        try:
            func = request.environ.get('werkzeug.server.shutdown')
            if func is None:
                raise RuntimeError('Not running with the Werkzeug Server')
            func()
        except BaseException as e:
            print(str(e))


    def get_all_functions(self, obj):
        """
        Gets all functions that were implemented.
        """
        try:
            all_functions = getmembers(obj, ismethod)
            if all_functions and len(all_functions) > 0:
                functions = [x[0] for x in all_functions if x[0] != '__init__']
                return functions
        except:
            return []


    def get_attr(self, obj, func_name):
        r"""Gets attributes by given function name.

        :param obj: object.
        :param func_name: method name of given object.

        :return: `Function`.
        """
        try:
            func = getattr(obj, func_name)
            return func
        except AttributeError:
            print("dostuff not found")

    def add_all_functions_to_rule(self, obj):
        r"""Binds all methods of object to server.

        :param obj: object.

        :return: `None`.
        """
        try:
            funcs = self.get_all_functions(obj=obj)
            # add all function
            self.add_url_rule('all_functions', view_func=lambda : str(funcs))
            for func in funcs:
                get_func = self.get_attr(obj=obj, func_name=func)
                if not self.add_url_rule(func, view_func=get_func):
                    self.is_err = True
                    self.err_msg = "could not bind function.."

        except BaseException as e:
            self.is_err = True
            self.err_msg = str(e)

    def get_request_data(self):
        return request.get_json()
    # def add_urls_rules(self, obj):
    #     for func in self.get_all_functions(obj=obj):
    #         self.add_url_rule(f)

if __name__=="__main__":
    s = Server_Cores()
    s.run()
    # from inspect import getmembers, ismethod, isfunction
    # a = [x[0] for x in getmembers(s, isfunction)]
    # print(getmembers(s, ismethod))



In [ ]:
#Agent/auto_thread.py
#
#import here
#
from cores.agent_server import Agent_Server


class Tasks():
    def __init__(self):
        pass

    def translate(txt='xin chào', sl='vie', dl='eng', ip='xx.xx.xx.xx', port=9000):
        args_data = Agent_Server.get_data()
        if args_data:
            txt = args_data['txt']
            sl = args_data['sl']
            dl = args_data['dl']
            ip = args_data['ip']
            port = args_data['port']
        #
        # code here
        #
        #
        # The return type must be a string, dict, tuple, Response instance, or WSGI callable
        return "this is translatation method"

    def answer(txt='', op='Com', ip='xx.xx.xx.xx', port=9000):
        args_data = Agent_Server.get_data()
        if args_data:
            txt = args_data['txt']
            op = args_data['op']
            ip = args_data['ip']
            port = args_data['port']
        #
        # code here
        #
        # The return type must be a string, dict, tuple, Response instance, or WSGI callable
        return "this is answer method"

    def search(txt='Hello', ip='xx.xx.xx.xx', port=9000):
        args_data = Agent_Server.get_data()
        if args_data:
            txt = args_data['txt']
            ip = args_data['ip']
            port = args_data['port']
        #
        # code here
        #
        # The return type must be a string, dict, tuple, Response instance, or WSGI callable
        return "this is search method"


In [ ]:
#Agent/test_client.py
from cores.agent_client import Agent_Client

if __name__ == '__main__':

    # creates `Client instance` object
    url = 'http://xx.xx.xx.xx:9000/'
    client = Agent_Client(url_path=url)

    input_text = """
        Txxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    """

    arr = []
    for i in range(50000):
        arr.append(i)
    # creates dictionary data which will pass to server functions as arguments
    data = {
            "txt":str(arr),
            "sl":"vie",
            "dl":"eng",
            "ip":"xx.xx.xx.xx",
            "port":9000
            }

    # shows all functions that can be requested
    # print(client.post(func_name='all_functions').text)

    # sends request to server to excute specific funtions
    rest = client.post(func_name='translate', data=data)
    print(rest.text)


In [ ]:
#Agent/test_server.py
from cores.install_packages import *
from cores.agent_server import *
from auto_thread import Tasks

if __name__ == "__main__":

    # creates `tasks instance` object that contain all functions of server
    # (ex: translate, answer, search).
    tasks = Tasks()

    # create server agent that will listen request from client.
    thread_agent = Agent_Server(
        ip='127.0.0.1',
        port=9000,
        obj=tasks)

    # runs agent
    thread_agent.run()

In [ ]:
#Vector_DB/embedding_class.py
from importlib.resources import path
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np
from langchain.vectorstores.faiss import FAISS
import os
from glob import glob
import pickle
# from langchain.text_splitter import CharacterTextSplitter

class NLP_Embedding():
    r"""NLP Embedding class that have some feature (Loading, Embedding, Saving,..)
        Base on LangChain Frame work & FAISS meta.
        """
    def __init__(self, embeded_model_path, db_local_path=None, is_load_local_model=False):
        r"""Creating new NLP_Embedding instance.

        :param embeded_model_path: `embedding folder path` (ex: ../embeddings/all_minilm_l6_v2/sentence-transformers_all-MiniLM-L6-v2).
        :param db_local_path: `local db` (raw db file path) (ex: ../db/source_sbert.db).
        :param is_load_local_model: select loading function from file or create new local db.

        :return: New instance.
        """
        self.model_path = embeded_model_path
        self.embeddings = HuggingFaceEmbeddings(model_name=embeded_model_path)
        if is_load_local_model:
            self.db = FAISS.load_local(db_local_path, self.embeddings)
        print('embeded model was loaded succesffuly~')

    def reload_local_model(self, db_local_path):
        r"""Reload model from local file path.

        :param db_local_path: `local db` (raw db file path) (ex: ../db/source_sbert.db).

        :return: new db.
        """
        self.db = FAISS.load_local(db_local_path, self.embeddings)

    def call(self, inputs):
        r"""Embeds inputs texts to embedding vectors.

        :param inputs: input texts.

        :return: new db.
        """
        self.db = FAISS.from_texts(inputs, self.embeddings)

    def save(self, saving_model_path):
        r"""Saves db to local files.

        :param saving_model_path: `local db` file path (ex: ../db/source_sbert.db).

        :return: void.
        """
        self.db.save_local(saving_model_path)
        print('db was saved.')

    def similarity_calculate(self, query, k: int = 4, with_score=False):
        r"""Return top k values that have best similarity score.

        :param query: searching value.
        :param k: return quantity.
        :param with_score: return score or not.

        :return: k values.

        :ex:
            `res = embedding.similarity_calculate(question, k=1, with_score=True)
            print(b.local_db[res[0][0].page_content]["VALUE"]))`
        """
        if with_score:
            return self.db.similarity_search_with_score(query, k)
        else:
            return self.db.similarity_search(query, k)

    def embed_from_dicts(self, dicts):
        r"""Embeds all keys of dictionary.

        :param saving_model_path: `local db` file path (ex: ../db/source_sbert.db).

        :return: void.
        """
        self.call(list(dicts.keys()))

    def embed_from_folder_dicts(self, input_folder, is_saved_embedded=True, saved_embedded_path=None, is_saved_dicts=True, saved_dicts_path=None):
        r"""Embeds all local dictionary files from folder.

        :param input_folder: including folder (ex: ../db/temp_db).
        :param is_saved_embedded: saves embedded model.
        :param saved_embedded_path: embedded model file path (ex: ../db/temp_db).
        :param is_saved_dicts: save dictionary db.
        :param saved_dicts_path: dictionary db file path (ex: ../db/dict.db).

        :return: void.
        """
        dicts = {}
        files = glob(os.path.join(input_folder, '*.db'))
        for file in files:
            with open(file, 'rb') as f:
                loaded_dict = pickle.load(f)
                dicts.update(loaded_dict)
        self.embed_from_dicts(dicts)

        if is_saved_embedded:
            self.save(saved_embedded_path)

        if is_saved_dicts:
            with open(saved_dicts_path, 'wb') as f:
                pickle.dump(dicts, f)
            print('db dicts was saved ~')


In [ ]:
#Vector_DB/local_db.py
from importlib.resources import path
import numpy as np
import pandas as pd
import requests
from os import path
import pickle
import sys, os
sys.path.append(os.getcwd())
from .embedding_class import NLP_Embedding

class Local_DB():
    r"""Local DB that is inclued all of local (dict) data.
        """
    def __init__(self, local_db_path=None, auto_load=True, show_head=False):
        r"""Init new Local DB instance.

        :param local_db_path: local db file path (eg: c:\...\).
        :param split_character: Split character is alway `\n.`

        :return: output cleaned `text`.
        """
        self.session = requests.Session()
        self.local_db = {}
        self.local_db_path = local_db_path
        if auto_load:
            if path.isfile(local_db_path):
                self.load_local_db(self.local_db_path)
            else:
                print("local db file is not existed. Please check..")

    def get_api_values(self, url, payload):
        response = self.session.post(url, data=payload)
        text = response.content.decode()
        tss = text
        return tss

    def load_local_db(self, local_db_path):
        db_frame = pd.read(local_db_path)
        sub_data = db_frame.loc[:, ['Item (En)', 'Detail']]
        self.local_db = sub_data.to_dict()
        print(self.local_db)

    def manual_insert_local_db(self):
        with open(self.local_db_path, 'r', encoding='utf-8') as _f:
                lines = _f.readlines()
                _is_first = True
                for line in lines:
                    if _is_first:
                        _is_first = False
                        continue

                    _s = line.split('\t')
                    self.local_db[_s[1]] = _s[2]

    def save_local_dict_db(self, save_path):
        """
            with open('saved_dictionary.pkl', 'wb') as f:
            pickle.dump(dictionary, f)

            with open('saved_dictionary.pkl', 'rb') as f:
            loaded_dict = pickle.load(f)
        """

        with open(save_path, 'wb') as f:
            pickle.dump(self.local_db, f)
        print("save success~")

    def load_local_dict_db(self, local_path):
        # loads dicts meta data
        with open(local_path, 'rb') as f:
            self.local_db = pickle.load(f)
        print('load local db successfuly~')

    def save_temp_dict_db(self, dict, save_path):
        """
            with open('saved_dictionary.pkl', 'wb') as f:
            pickle.dump(dictionary, f)

            with open('saved_dictionary.pkl', 'rb') as f:
            loaded_dict = pickle.load(f)
        """

        with open(save_path, 'wb') as f:
            pickle.dump(dict, f)

if __name__ == "__main__":
    embedding_path = r'xx\all_minilm_l6_v2\sentence-transformers_all-MiniLM-L6-v2'
    db_save_embed_path = r'xx\embeddings\db\source_sbert.db'
    db_save_dict_path = r'xx\embeddings\db\source_sbert.db\dicts.db'

    embedding = NLP_Embedding(embeded_model_path=embedding_path, db_local_path=db_save_embed_path, is_load_local_model=False)
    embedding.embed_from_folder_dicts(r'xx\embeddings\db\temp_db', saved_embedded_path=db_save_embed_path, saved_dicts_path=db_save_dict_path)
    res = embedding.similarity_calculate("Hai Duong bus", 1, True)
    print(res)

    # local_db.manual_insert_local_db()
    # local_db.save_local_dict_db(r'xx\embeddings\db\source_sbert.db\dicts.db')
    # embedding.call(local_db.local_db.keys())
    # embedding.save(db_save_path)

    # payload = {
    # 	"NAME": "big_sql",
    #     "lang": "VN",
    #     "RPTCODEID": "TEXT_19",
    #     "DB": "MB3EDW",
    #     "keys": "",
    #     "values": ""
    # }
    # url_server = 'http://xx.xx.xx.xx:8081/Chatbot/runSqlRPT'
    # local_db = Local_DB()
    # values = local_db.get_api_values(url_server, payload)
    # print(values)

In [ ]:
#auto_thread.py
#
#import here
#
from Agent.cores.agent_server import Agent_Server
from Vector_DB.embedding_class import NLP_Embedding

class Tasks():
    def __init__(self, embedding_path, db_save_embed_path, is_load_local_model=True):
        self.embedding = NLP_Embedding(embeded_model_path=embedding_path, db_local_path=db_save_embed_path, is_load_local_model=is_load_local_model)
        print("embeded db was created.")

    def embedding(self, arrays=[], is_save=True, save_path=""):
        args_data = Agent_Server.get_data()
        arrays = args_data['arrays']
        self.embedding.call(arrays)
        if is_save:
            self.embedding.save(save_path)
        return "ok"

    def similarity_search(self, txt=""):
        args_data = Agent_Server.get_data()
        txt = args_data['txt']
        res = self.embedding.similarity_calculate(txt, 1, True)
        return {"content":res[0][0].page_content, "distance":str(res[0][1])}


    def translate(txt='xin chào', sl='vie', dl='eng', ip='xx.xx.xxx.xx', port=9000):
        args_data = Agent_Server.get_data()
        if args_data:
            txt = args_data['txt']
            sl = args_data['sl']
            dl = args_data['dl']
            ip = args_data['ip']
            port = args_data['port']
        #
        # code here
        #
        #
        # The return type must be a string, dict, tuple, Response instance, or WSGI callable
        return "this is translatation method"

    def answer(txt='', op='Com', ip='xx.xx.xx.xx', port=9000):
        args_data = Agent_Server.get_data()
        if args_data:
            txt = args_data['txt']
            op = args_data['op']
            ip = args_data['ip']
            port = args_data['port']
        #
        # code here
        #
        # The return type must be a string, dict, tuple, Response instance, or WSGI callable
        return "this is answer method"

    def search(txt='Hello', ip='xx.xx.xx.xx', port=9000):
        args_data = Agent_Server.get_data()
        if args_data:
            txt = args_data['txt']
            ip = args_data['ip']
            port = args_data['port']
        #
        # code here
        #
        # The return type must be a string, dict, tuple, Response instance, or WSGI callable
        return "this is search method"

In [ ]:
#main.py
from Agent.cores.install_packages import *
from Agent.cores.agent_server import *
from auto_thread import Tasks

if __name__ == "__main__":

    # creates `tasks instance` object that contain all functions of server
    # (ex: translate, answer, search).
    embedding_path = r'xx\all_minilm_l6_v2\sentence-transformers_all-MiniLM-L6-v2'
    db_save_embed_path = r'xx\embeddings\temp.db'
    tasks = Tasks(embedding_path, db_save_embed_path)

    # create server agent that will listen request from client.
    thread_agent = Agent_Server(
        ip='127.0.0.1',
        port=9000,
        obj=tasks)

    # runs agent
    thread_agent.run()

In [ ]:
#test.py
from Agent.cores.agent_client import Agent_Client

if __name__ == '__main__':

    # creates `Client instance` object
    url = 'http://127.0.0.1:9000/'
    client = Agent_Client(url_path=url)

    # creates dictionary data which will pass to server functions as arguments
    data = {
            "txt":"rightt click"
            }

    # shows all functions that can be requested
    # print(client.post(func_name='all_functions').text)

    # sends request to server to excute specific funtions
    rest = client.post(func_name='similarity_search', data=data)
    print(rest.text)



In [ ]:
#c#
using System.Net;
using System.Net.Http.Headers;
using System.Text;
using Newtonsoft.Json;

string url = "http://127.0.0.1:9000/similarity_search";
var dict = new Dictionary<string,string>();
dict["txt"] = "right click";
var json_dict = JsonConvert.SerializeObject(dict);
var content = new StringContent(json_dict, Encoding.UTF8, "application/json");
var client = new HttpClient();
var request = await client.PostAsync(url, content);
var contents = await request.Content.ReadAsStringAsync();
Console.WriteLine(contents);